In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from base import BaseModel
from trainer import *
from model import *
from sklearn.datasets import make_classification
from sklearn import svm
from sklearn.model_selection import train_test_split
confined_b = Confined_bias.apply
confined_w = Confined_weight.apply

In [44]:
layers=[20, 8, 3]

In [24]:
model = Model(*layers)
# 확인하고 싶은 Model 불러오기
loading = torch.load('saved/models/gene_net_20/0111_000619/model_best.pth', map_location='cpu')
sd = loading['state_dict']
model.load_state_dict(sd)
model.eval()

weights={}
biases={}
for i in range(len(layers)-1):
    weights['weight'+str(i+1)]=confined_w(sd['fc'+str(i+1)+'.weight'])
    biases['bias'+str(i+1)]=confined_b(sd['fc'+str(i+1)+'.bias'])
    print(sum(sum(weights['weight'+str(i+1)]!=0)))
    print(weights['weight'+str(i+1)])
    print(sum(biases['bias'+str(i+1)]!=0))
    print(biases['bias'+str(i+1)])
print()
for i in range(len(layers)-1):
    weights['weight'+str(i+1)+'_t']=torch.transpose(weights['weight'+str(i+1)],0,1)
    print(weights['weight'+str(i+1)+'_t'])

tensor(127)
tensor([[ 0.0000,  0.3000, -0.1000, -0.1000, -0.1000,  0.1000, -0.2000, -0.1000,
          0.2000,  0.4000,  0.0000, -0.2000, -0.2000, -0.4000, -0.2000,  0.1000,
         -0.2000,  0.3000, -0.0000,  0.0000],
        [-0.2000, -0.0000, -0.1000,  0.2000, -0.4000,  0.3000,  0.1000, -0.1000,
         -0.3000, -0.6000, -0.1000,  0.3000, -0.1000,  0.3000, -0.0000,  0.0000,
         -0.4000, -0.5000, -0.1000, -0.5000],
        [ 0.3000,  0.1000, -0.2000,  0.2000,  0.1000, -0.2000, -0.2000,  0.0000,
          0.2000,  0.5000,  0.2000, -0.1000,  0.3000, -0.1000,  0.3000, -0.3000,
          0.2000,  0.6000,  0.0000,  0.3000],
        [-0.2000, -0.0000, -0.4000, -0.3000, -0.3000, -0.2000, -0.3000, -0.2000,
         -0.4000, -0.1000, -0.0000, -0.1000, -0.3000, -0.4000, -0.0000, -0.2000,
          0.0000, -0.4000, -0.2000,  0.1000],
        [-0.0000, -0.1000, -0.1000,  0.0000, -0.3000, -0.1000, -0.2000,  0.2000,
          0.0000, -0.0000, -0.3000,  0.2000, -0.1000,  0.2000, -0.2000, -0.

In [25]:
data_dir='../'
rna_exp = pd.read_csv(data_dir+'/tiny_data_'+str(layers[0])+'.tsv', sep='\t')
labels = pd.read_csv(data_dir+'/tiny_label_'+str(layers[0])+'.tsv', sep='\t')
X = rna_exp
y_ = labels['cancer type']
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
lb.fit(y_)
y=lb.transform(y_)

X=torch.tensor(X.to_numpy(), dtype=torch.float32)
answer=torch.tensor(y, dtype=torch.long)
results={}
results['s0']=torch.tensor(X, dtype=torch.float)
for i in range(2*len(layers)-3):
    if i%2==0:
        results['s'+str(i+1)]=torch.mm(results['s'+str(i)], weights['weight'+str(i//2+1)+'_t'])+biases['bias'+str(i//2+1)]
    else :
        results['s'+str(i+1)]=F.relu(results['s'+str(i)])
print(str((torch.argmax(results['s'+str(2*len(layers)-3)], dim=1)==answer).float().sum().numpy())+'/'+str(answer.size(0)))
print((torch.argmax(results['s'+str(2*len(layers)-3)], dim=1)==answer).float().sum().numpy()/answer.size(0))


1589.0/1600
0.993125


<ipython-input-25-4a0126c90fce>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  results['s0']=torch.tensor(X, dtype=torch.float)


In [41]:
# for SVM
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=42)

In [42]:
clf = svm.SVC()
clf.fit(X_train, y_train)

SVC()

In [43]:
# SVM accuracy
sum(y_test==clf.predict(X_test))/len(y_test)

0.9583333333333334

In [34]:
torch.max(results['s3'])

tensor(175662.5000)

In [12]:
layers=[3, 4, 2]

In [13]:
model = Model(*layers)
# 확인하고 싶은 Model 불러오기
loading = torch.load('saved/models/sklearn_net/clinear_4_2_scaled_bt_1/model_best.pth', map_location='cpu')
sd = loading['state_dict']
model.load_state_dict(sd)
model.eval()

weights={}
biases={}
for i in range(len(layers)-1):
    weights['weight'+str(i+1)]=confined_w(sd['fc'+str(i+1)+'.weight'])
    biases['bias'+str(i+1)]=confined_b(sd['fc'+str(i+1)+'.bias'])
    print(sum(sum(weights['weight'+str(i+1)]!=0)))
    print(weights['weight'+str(i+1)])
    print(sum(biases['bias'+str(i+1)]!=0))
    print(biases['bias'+str(i+1)])
print()
for i in range(len(layers)-1):
    weights['weight'+str(i+1)+'_t']=torch.transpose(weights['weight'+str(i+1)],0,1)
    print(weights['weight'+str(i+1)+'_t'])

tensor(11)
tensor([[-0.2000,  0.0000, -0.2000],
        [ 0.3000, -0.4000,  0.2000],
        [-0.3000, -0.4000, -0.3000],
        [ 0.7000,  0.7000, -0.3000]])
tensor(2)
tensor([ 0.0000, -0.2260,  0.0000, -0.5100])
tensor(8)
tensor([[-0.4000, -0.3000, -0.2000, -0.7000],
        [-0.4000,  0.3000, -0.1000,  0.7000]])
tensor(1)
tensor([ 0.0000, -0.3390])

tensor([[-0.2000,  0.3000, -0.3000,  0.7000],
        [ 0.0000, -0.4000, -0.4000,  0.7000],
        [-0.2000,  0.2000, -0.3000, -0.3000]])
tensor([[-0.4000, -0.4000],
        [-0.3000,  0.3000],
        [-0.2000, -0.1000],
        [-0.7000,  0.7000]])


In [14]:
pre_X, y = make_classification(n_samples=2000, n_classes=2, n_informative=3, n_redundant=0, 
                    n_features=3, random_state=123)
# from sklearn import preprocessing
# X_normalized = preprocessing.normalize(pre_X, norm='l2')*10
X=pre_X + abs(np.min(pre_X)) + np.array([1,1,1])
X=np.around(X*0.1, 4)
if len(np.unique(X, axis=0))==y.size:
    print('중복되는 값 없음!')

answer=torch.tensor(y, dtype=torch.long)
results={}
results['s0']=torch.tensor(X, dtype=torch.float)
for i in range(2*len(layers)-3):
    if i%2==0:
        results['s'+str(i+1)]=torch.mm(results['s'+str(i)], weights['weight'+str(i//2+1)+'_t'])+biases['bias'+str(i//2+1)]
    else :
        results['s'+str(i+1)]=F.relu(results['s'+str(i)])
print(str((torch.argmax(results['s'+str(2*len(layers)-3)], dim=1)==answer).float().sum().numpy())+'/'+str(answer.size(0)))
print((torch.argmax(results['s'+str(2*len(layers)-3)], dim=1)==answer).float().sum().numpy()/answer.size(0))


중복되는 값 없음!
1640.0/2000
0.82


In [15]:
ck=0
for i in range(results['s'+str(2*len(layers)-3)].size(0)):
    if results['s'+str(2*len(layers)-3)][i][0]<=0 and results['s'+str(2*len(layers)-3)][i][1]<=0:
        ck+=1
print(ck)

1947
